In [1]:
# See description above for what these variables represent
big_loop_iterations = 10
explainer_runs = 10
thresholds = [30, 50]
samples = [10]

# Used to label the resulting files -
date = "quantile_alg_final"

# Set up directory for result files
import os
dir = f'./results_{date}'
if not os.path.exists(dir):
    os.mkdir(dir)

In [2]:
from GNNSubNet import GNNSubNet as gnn
import pandas as pd
import numpy as np

# # Kidney data set  ------------------------- #
loc   = "../TCGA"
ppi   = f'{loc}/KIDNEY_RANDOM_PPI.txt'
feats = [f'{loc}/KIDNEY_RANDOM_Methy_FEATURES.txt', f'{loc}/KIDNEY_RANDOM_mRNA_FEATURES.txt']
targ  = f'{loc}/KIDNEY_RANDOM_TARGET.txt'

# # Synthetic data set  ------------------------- #
# loc   = "../GNNSubNet/datasets/synthetic/"
# ppi   = f'{loc}/NETWORK_synthetic.txt'
# feats = [f'{loc}/FEATURES_synthetic.txt']
# targ  = f'{loc}/TARGET_synthetic.txt'

# Read in the synthetic data
g = gnn.GNNSubNet(loc, ppi, feats, targ, normalize=False)

# Get some general information about the data dimension
g.summary()

Graph is connected  False
Calculate subgraph ...
Number of subgraphs:  118
Size of subgraph:  2049
Graph is connected  True
##################
# DATASET LOADED #
##################

Number of nodes: 2049
Number of edges: 13588
Number of modalities: 2


In [3]:
def obtain_BAGEL_scores(loc, ppi, feats, targ, gnn_subnet, quantile_aggregation=False, quantile=0.5):
    model_info = []
    fidelity = []
    validity_plus = []
    validity_minus = []
    validity_plus_matrix = []
    validity_minus_matrix = []
    sparsity = []

    for i in range(9, big_loop_iterations):
        print(i)
        g = gnn.GNNSubNet(loc, ppi, feats, targ, normalize=False)
        g.train()

        # Check the performance of the classifier
        accuracy = g.accuracy

        # Run the explainer the desired number of times
        g.explain(explainer_runs, gnn_subnet=gnn_subnet, quantile_aggregation=quantile_aggregation, quantile=quantile)

        # Fidelity
        f = g.evaluate_RDT_fidelity_soft()
        fidelity.append([i, accuracy, np.mean(f)])
        # Save mean fidelity for each sample for further analysis
        filename = f"results_{date}/{i}_fidelities.csv"
        np.savetxt(filename, fidelity, delimiter=',', fmt ='% s')

        # Sparsity
        sparsities = g.evaluate_sparsity()
        # Save raw results in case needed for further analysis
        filename = f"results_{date}/{i}_sparsities.csv"
        np.savetxt(filename, sparsities, delimiter=',', fmt ='% s')
        # Save mean sparsity to list to create processed table
        sparsity.append([i, accuracy, np.mean(sparsities)])

        # Validity at varying thresholds
        for t in thresholds:
            v_plus, v_minus, mat_plus, mat_minus = g.evaluate_validity(threshold=t, confusion_matrix=True)
            validity_plus.append([i, accuracy, t, v_plus])
            validity_minus.append([i, accuracy, t, v_minus])
            validity_plus_matrix.append([i, accuracy, t, mat_plus[0,0], mat_plus[0,1], mat_plus[1,0], mat_plus[1,1]])
            validity_minus_matrix.append([i, accuracy, t, mat_minus[0,0], mat_minus[0,1], mat_minus[1,0], mat_minus[1,1]])

        filename = f"results_{date}/{i}_validity_plus.csv"
        np.savetxt(filename, validity_plus, delimiter=',', fmt ='% s')

        filename = f"results_{date}/{i}_validity_minus.csv"
        np.savetxt(filename, validity_minus, delimiter=',', fmt ='% s')

In [4]:
print(f"results_{date}")

results_quantile_alg_final


In [5]:
# Runs the experiment for a single explainer
# Toggle the gnn_subnet parameter for the desired explainer
obtain_BAGEL_scores(loc, ppi, feats, targ, gnn_subnet=False, quantile_aggregation=True, quantile=0.5)

9
Graph is connected  False
Calculate subgraph ...
Number of subgraphs:  118
Size of subgraph:  2049
Graph is connected  True
##################
# DATASET LOADED #
##################
graphcnn for training ...
Graphs class 0: 200, Graphs class 1: 306
Length of balanced dataset list: 400
Train graph class 0: 162, train graph class 1: 158
Validation graph class 0: 38, validation graph class 1: 42


  0%|          | 0/35 [00:00<?, ?batch/s]C:\Users\elena\DELFT\assignments\RP\GNN-SubNet_modified\GNNSubNet\graphcnn.py:134: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:623.)
  graph_pool = torch.sparse.FloatTensor(idx, elem, torch.Size([len(batch_graph), start_idx[-1]]))
100%|██████████| 35/35 [00:08<00:00,  4.10batch/s]


Epoch 0, loss 2190.6757
Train Acc 0.5156
Epoch 0, val_loss 24.9896
Saving best model with validation loss 24.989639282226562


100%|██████████| 35/35 [00:11<00:00,  3.10batch/s]


Epoch 1, loss 332.4395
Train Acc 0.5062
Epoch 1, val_loss 185.1322


...

Epoch 18, loss 15.5479
Train Acc 0.4969
Epoch 18, val_loss 29.1389


100%|██████████| 35/35 [00:06<00:00,  5.77batch/s]


Epoch 19, loss 19.2215
Train Acc 0.7750
Epoch 19, val_loss 4.0681

Confusion matrix (Validation set):

[[25 13]
 [ 9 33]]
Validation accuracy: 72.5%
Validation loss 3.351222276687622

------- Run the Explainer -------

GNN-SubNet: False
Explainer::Iteration 1 of 10
Explainer::Iteration 2 of 10
Explainer::Iteration 3 of 10
Explainer::Iteration 4 of 10
Explainer::Iteration 5 of 10
Explainer::Iteration 6 of 10
Explainer::Iteration 7 of 10
Explainer::Iteration 8 of 10
Explainer::Iteration 9 of 10
Explainer::Iteration 10 of 10
Evaluating RDT-fidelity using 80 graphs from the test dataset
Evaluating 1 out of 80


C:\Users\elena\DELFT\assignments\RP\GNN-SubNet_modified\GNNSubNet\GNNSubNet.py:1715: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_graph = Data(x=torch.tensor(perturbed_features).float().to("cpu"),


Evaluating 2 out of 80
Evaluating 3 out of 80
Evaluating 4 out of 80
Evaluating 5 out of 80
Evaluating 6 out of 80
Evaluating 7 out of 80
Evaluating 8 out of 80
Evaluating 9 out of 80
Evaluating 10 out of 80
Evaluating 11 out of 80
Evaluating 12 out of 80
Evaluating 13 out of 80
Evaluating 14 out of 80
Evaluating 15 out of 80
Evaluating 16 out of 80
Evaluating 17 out of 80
Evaluating 18 out of 80
Evaluating 19 out of 80
Evaluating 20 out of 80
Evaluating 21 out of 80
Evaluating 22 out of 80
Evaluating 23 out of 80
Evaluating 24 out of 80
Evaluating 25 out of 80
Evaluating 26 out of 80
Evaluating 27 out of 80
Evaluating 28 out of 80
Evaluating 29 out of 80
Evaluating 30 out of 80
Evaluating 31 out of 80
Evaluating 32 out of 80
Evaluating 33 out of 80
Evaluating 34 out of 80
Evaluating 35 out of 80
Evaluating 36 out of 80
Evaluating 37 out of 80
Evaluating 38 out of 80
Evaluating 39 out of 80
Evaluating 40 out of 80
Evaluating 41 out of 80
Evaluating 42 out of 80
Evaluating 43 out of 80
